# **Importing Data**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense

2024-06-12 17:44:33.192961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 17:44:33.193096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 17:44:33.378751: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
users_df = pd.read_csv("/kaggle/input/users-csv/Users.csv")
ratings_df = pd.read_csv("/kaggle/input/ratings-csv/Ratings.csv")
books_df = pd.read_csv("/kaggle/input/books-csv/Books.csv")

/tmp/ipykernel_18/1342344810.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("/kaggle/input/books-csv/Books.csv")


In [3]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


# **Preprocessing Data**

In [6]:
# Dropping Image Columns
books_df.drop(books_df.columns[5:], axis=1, inplace=True)

In [7]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [8]:
# Merge datasets
merged_df = pd.merge(ratings_df, books_df, on='ISBN')
final_df = pd.merge(merged_df, users_df, on='User-ID')

# Drop NA values
final_df.dropna(inplace=True)

# Replace non-numeric values with '0'
final_df['Year-Of-Publication'] = final_df['Year-Of-Publication'].replace(['Gallimard', 'DK Publishing Inc'], '0')

# Convert specific columns to integers
final_df['User-ID'] = final_df['User-ID'].astype(int)
final_df['Book-Rating'] = final_df['Book-Rating'].astype(int)
final_df['Age'] = final_df['Age'].astype(int)

# Convert Year-Of-Publication column to integers
final_df['Year-Of-Publication'] = final_df['Year-Of-Publication'].astype(int)

# Convert specific columns to strings
final_df['ISBN'] = final_df['ISBN'].astype(str)
final_df['Book-Title'] = final_df['Book-Title'].astype(str)
final_df['Book-Author'] = final_df['Book-Author'].astype(str)
final_df['Publisher'] = final_df['Publisher'].astype(str)
final_df['Location'] = final_df['Location'].astype(str)

# Define which columns are categorical and numerical
categorical_cols = ['ISBN', 'Book-Title', 'Book-Author', 'Publisher', 'Location']
numerical_cols = ['User-ID', 'Age', 'Year-Of-Publication']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

In [9]:
# Split data into train and test sets
X = final_df.drop(['Book-Rating'], axis=1)
y = final_df['Book-Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Fit and transform preprocessing on training data, and transform on test data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# **Building The Tensorflow Model**

In [11]:
# Define and compile the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='relu')
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
history = model.fit(X_train_preprocessed, y_train, epochs=10, validation_split=0.2, batch_size=528)

# Evaluate the model
mse, _ = model.evaluate(X_test_preprocessed, y_test)
print("Mean Squared Error:", mse)





/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 246s 267ms/step - loss: 14.6232 - mean_squared_error: 14.6232 - val_loss: 12.3537 - val_mean_squared_error: 12.3537
Epoch 2/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 246s 269ms/step - loss: 9.0676 - mean_squared_error: 9.0676 - val_loss: 13.0558 - val_mean_squared_error: 13.0558
Epoch 3/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 247s 270ms/step - loss: 6.4749 - mean_squared_error: 6.4749 - val_loss: 13.5926 - val_mean_squared_error: 13.5926
Epoch 4/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 247s 271ms/step - loss: 5.0737 - mean_squared_error: 5.0737 - val_loss: 14.4576 - val_mean_squared_error: 14.4576
Epoch 5/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 248s 271ms/step - loss: 4.1592 - mean_squared_error: 4.1592 - val_loss: 14.6065 - val_mean_squared_error: 14.6065
Epoch 6/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 249s 272ms/step - loss: 3.4818 - mean_squared_error: 3.4818 - val_loss: 15.1312 - val_mean_squared_error: 15.1312
Epoch 7/10
914/914 ━━━━━━━━━━━━━━━━━━━━ 250s 274ms/step - loss: 2.9585 - m

In [12]:
model.save('tf_model.h5')